In [1]:
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent / "src"))
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from config import *
from train import train_one_epoch
from evaluate import evaluate
from predict import predict_folder

In [2]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
train_dataset = datasets.ImageFolder(DATA_DIR / "train", transform=train_transform)
val_dataset   = datasets.ImageFolder(DATA_DIR / "val", transform=val_test_transform)
test_dataset  = datasets.ImageFolder(DATA_DIR / "test", transform=val_test_transform)

train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader   = DataLoader(val_dataset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader  = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

class_names = train_dataset.classes
num_classes = len(class_names)

In [4]:
with open(CLASSES_PATH, "w", encoding="utf-8") as f:
    for c in class_names:
        f.write(c + "\n")

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

C:\Users\admin\miniconda3\envs\pytorch_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\admin\miniconda3\envs\pytorch_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

In [7]:
SAVED_MODELS_DIR.mkdir(exist_ok=True)
best_val_acc = 0.0

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device
    )
    val_loss, val_acc = evaluate(
        model, val_loader, criterion, device
    )

    print(f"[{epoch+1}/{EPOCHS}] "
          f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), SAVED_MODELS_DIR / "best_model.pt")

[1/5] Train Acc: 0.8839 | Val Acc: 0.9442
[2/5] Train Acc: 0.9440 | Val Acc: 0.9471
[3/5] Train Acc: 0.9490 | Val Acc: 0.9485
[4/5] Train Acc: 0.9528 | Val Acc: 0.9550
[5/5] Train Acc: 0.9569 | Val Acc: 0.9600


In [8]:
best_model = models.resnet18(pretrained=False)
best_model.fc = nn.Linear(best_model.fc.in_features, num_classes)

best_model.load_state_dict(
    torch.load(SAVED_MODELS_DIR / "best_model.pt", map_location=device)
)
best_model = best_model.to(device)

test_loss, test_acc = evaluate(best_model, test_loader, criterion, device)
print("Test Accuracy:", test_acc)

C:\Users\admin\miniconda3\envs\pytorch_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\admin\AppData\Local\Temp\ipykernel_17344\3833950088.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

Test Accuracy: 0.9392857142857143


In [9]:
results = predict_folder(
    best_model,
    PREDICT_DIR,
    class_names,
    val_test_transform,
    device,
    topk=3
)

for img, preds in results.items():
    print(f"\n{img}")
    for label, prob in preds:
        print(f"  {label}: {prob:.4f}")


butter01.jpg
  butterfly: 0.9698
  cat: 0.0204
  spider: 0.0053

cat01.jpg
  cat: 1.0000
  butterfly: 0.0000
  spider: 0.0000

chicken01.jpg
  chicken: 0.9993
  butterfly: 0.0006
  horse: 0.0001

cow01.jpg
  cow: 0.9867
  horse: 0.0112
  sheep: 0.0020

dog01.jpg
  dog: 0.9921
  sheep: 0.0071
  cow: 0.0004

dog02.jpg
  dog: 0.9497
  sheep: 0.0131
  horse: 0.0127

dog03.jpg
  dog: 0.9899
  cat: 0.0094
  butterfly: 0.0006

elephant01.jpg
  elephant: 1.0000
  sheep: 0.0000
  horse: 0.0000

spider01.jpg
  spider: 1.0000
  butterfly: 0.0000
  monkey: 0.0000
